## Machine Learning approach to contrast the manual selection and recommendation problem

The objective of this model will be to assess the probability of the group of selected people to engage or not by purchasing an em account.

For that I will use the following data:
- To be consistent, only customers that have been considered active this last month will be considered.
- TARGET will be em account.
- Distribution of products that a given customer has in last partition (except em account)
- revenue computed in last manual step
- months being active

This model is extremely primitive and thus very limited. 
The final goal is to predict the probability of a client that has no products, purchasing an em account. In order to do so it will only have access to a list of products == 0 and the number of months which that client has been active in the app and 3 values of revenue (0,10,20) but mostly 0.
In order to reach this goal the model will have to learn patterns from clients' products list (containing ones), number of months (most consistent feature across the dataset) and values of revenue (greater than zero) which will lead to not very reliable results.

About looking for similar profiles in other months, again the limitation is the features we are inputting. If anything the only thing that will enrich the model are demographics:
- Age range for example
- Entry channel..
- Gender at some point...

## Libraries

In [2]:
import pandas as pd
import numpy as np

## VISUALIZATION
import matplotlib.pyplot as plt
import seaborn as sns

## SKLEARN
import sklearn
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

## XGBOOST
import xgboost as xgb
from xgboost import cv
from xgboost import XGBClassifier

## SHAP
import shap

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

## Model

In [5]:
## load data

model_df = pd.read_csv('model_proba_df.csv', index_col=0)
model_df.head()

,revenue,months,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,debit_card,...,em_account_p,em_acount,age_u18,age_18-30,age_31-40,age_41-50,age_51-60,age_61-70,age_71-80,age_o80
1128353,5050,17,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
1116675,4980,17,0,0,0,1,1,0,1,1,...,0,1,0,0,0,0,0,1,0,0
1136671,4840,17,0,0,0,1,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
1070525,4610,17,0,0,0,1,1,1,0,1,...,0,0,0,0,0,1,0,0,0,0
1053446,4590,17,0,0,0,1,0,1,1,1,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
## Data partition and sampling
# First: remove our 10k customers from database because they will be "new data" to contrast the model when ready

final_10k = 